# Azure AI Agent ServiceとSemantic Kernelを使用したRAG

このコードスニペットは、`Azure AI Agent Service`と`Semantic Kernel`を使用して、検索強化生成（RAG）のためのAzure AIエージェントを作成および管理する方法を示しています。エージェントは取得したコンテキストに基づいてユーザーのクエリを処理し、それに応じて正確な回答を提供します。


SQLite バージョン修正  
次のエラーが発生した場合:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

ノートブックの冒頭でこのコードブロックのコメントを解除してください:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### パッケージのインポート
以下のコードは必要なパッケージをインポートします:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# セマンティックカーネルとAzure AIエージェントサービスを用いた検索強化生成 (RAG)

このサンプルでは、**Azure AIエージェントサービス**を使用して、言語モデルとアップロードされたドキュメントからのドメイン固有のコンテキストを組み合わせることで、**検索強化生成 (RAG)** を実行する方法を示します。

### 仕組み

1. **ドキュメントのアップロード**: 情報（Contosoの旅行保険ポリシー）を含むMarkdownファイル (document.md) をエージェントサービスにアップロードします。

2. **ベクトルストアの作成**: ドキュメントがベクトルストアにインデックス化され、その内容に対してセマンティック検索が可能になります。

3. **エージェントの設定**: 以下の厳密な指示を伴い、`gpt-4o`モデルを使用してエージェントを構成します:
   - ドキュメントから取得した内容に基づいてのみ質問に回答する。
   - 質問が範囲外の場合は回答を控える。

4. **ファイル検索ツールの統合**: エージェントに`FileSearchTool`を登録し、推論中にインデックス化されたドキュメントから関連するスニペットを検索・取得できるようにします。

5. **ユーザーとの対話**: ユーザーが質問を行います。ドキュメント内に関連情報が見つかった場合、エージェントは根拠のある回答を生成します。  
   見つからない場合、エージェントはドキュメントに十分な情報が含まれていないことを明確に回答します。


### メイン関数

このセクションでは、プログラムのメイン関数について説明します。

#### 概要
メイン関数は、アプリケーションのエントリーポイントとして機能します。すべての初期化処理を行い、主要なロジックを実行します。

#### 使用方法
以下のコードスニペットは、メイン関数の基本的な構造を示しています。

```python
def main():
    initialize()
    execute_main_logic()
    finalize()
```

- `initialize()`: 必要なリソースや設定を初期化します。
- `execute_main_logic()`: アプリケーションの主要な処理を実行します。
- `finalize()`: 終了時のクリーンアップ処理を行います。

#### 注意事項
[!IMPORTANT] メイン関数内で例外処理を適切に実装してください。これにより、予期しないエラーが発生した場合でも、プログラムが安全に終了します。

#### サンプルコード
以下は、メイン関数の実装例です。

```python
def main():
    try:
        initialize()
        execute_main_logic()
    except Exception as e:
        handle_error(e)
    finally:
        finalize()
```

- `handle_error(e)`: エラーをログに記録し、必要に応じて通知を送信します。

#### よくある質問
**Q: メイン関数を複数作成することはできますか？**  
A: 通常、メイン関数は1つだけです。ただし、特定の条件下で異なるエントリーポイントを作成することは可能です。

**Q: メイン関数内でグローバル変数を使用しても問題ありませんか？**  
A: グローバル変数の使用は避けるべきです。代わりに、関数間でデータを渡す方法を検討してください。

#### まとめ
メイン関数は、アプリケーションの中心的な役割を果たします。適切に設計し、エラー処理を含めることで、信頼性の高いプログラムを構築できます。


Azure CLI を使用して適切な認証コンテキストが提供されるように、最初に `az login` を実行してください。Azure AI Agent Service は API キーを使用しません。


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

了解しました。以下に翻訳を提供します。

---

# はじめに

このドキュメントでは、@@INLINE_CODE_0@@を使用してアプリケーションを構築する方法について説明します。@@INLINE_CODE_1@@は、開発者が効率的に作業できるように設計されたツールです。

[!NOTE] このガイドは初心者向けです。経験豊富な開発者は、特定のセクションをスキップしても問題ありません。

---

## 必要条件

以下の項目が必要です:

- 最新バージョンの@@INLINE_CODE_2@@
- @@INLINE_CODE_3@@がインストールされた環境
- 基本的なプログラミング知識

[!WARNING] 必要条件を満たしていない場合、エラーが発生する可能性があります。

---

## インストール

1. @@INLINE_CODE_4@@をダウンロードします。
2. 次のコマンドを実行してインストールします:

   ```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

3. インストールが完了したら、以下のコマンドで動作確認を行います:

   @@CODE_BLOCK_1@@

[!TIP] インストール中に問題が発生した場合は、公式ドキュメントを参照してください。

---

## 基本的な使い方

以下のコードスニペットは、@@INLINE_CODE_5@@を使用して簡単なアプリケーションを作成する方法を示しています:

@@CODE_BLOCK_2@@

[!IMPORTANT] このコードはサンプルです。実際のプロジェクトでは適宜変更してください。

---

## よくある問題

### 問題: インストールが失敗する

解決策:
- @@INLINE_CODE_6@@が正しく設定されていることを確認してください。
- 必要な権限があるか確認してください。

[!CAUTION] 権限を変更する際は慎重に行ってください。

---

## 次のステップ

これで基本的なセットアップは完了です。次のステップでは、@@INLINE_CODE_7@@を使用してプロジェクトを拡張する方法を学びます。

公式ドキュメントはこちら: @@INLINE_CODE_8@@

---

以上が翻訳内容です。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は責任を負いません。
